In this demo, we will show how you can use:

1. `ActionSet` to specify how a consumer can alter the input variables of a machine learning model

2. `Flipset` to generate an actionable adverse action notice for a consumer who is denied a loan

3. `RecourseAuditor` to measure the feasibility and difficulty of recourse for a given model before it is deployed



# Preliminaries

You can install using `pip`

In [1]:
! pip install actionable-recourse

     |████████████████████████████████| 46 kB 1.1 MB/s 
You should consider upgrading via the '/Users/berk/.pyenv/versions/3.7.8/bin/python3.7 -m pip install --upgrade pip' command.


We start by building a simple logistic regression for loan approval for the demo. We train the model using the **default of credit card clients** dataset from the [UCI ML repository](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients).


In [2]:
import recourse as rs
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from IPython.core.display import display, HTML
pd.options.display.float_format = '{:,.3f}'.format

# import data
url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
df = pd.read_csv(url)
y, X = df.iloc[:, 0], df.iloc[:, 1:]

# train a classifier
clf = LogisticRegression(max_iter = 1000)
clf.fit(X, y)
yhat = clf.predict(X)

The outcome variable is defined as:

> y[i] = 1 if a person repays a credit card loan

> y[i] = 0 if a person defaults on a credit card loan

The model `clf` predicts this outcome using the following input variables and weights


In [3]:
pd.Series(clf.coef_[0], index=X.columns).to_frame('Coefficients')

,Coefficients
Married,0.371
Single,0.576
Age_lt_25,0.208
Age_in_25_to_40,0.436
Age_in_40_to_59,0.323
Age_geq_60,0.011
EducationLevel,0.002
MaxBillAmountOverLast6Months,0.000
MaxPaymentAmountOverLast6Months,0.000
MonthsWithZeroBalanceOverLast6Months,0.064


# Using `ActionSet` to specify how each variable can be changed

The first step of any recourse-related analysis is to specify how a person can change the input variables to the model. This can quickly become tedious – especially when models contain tens or hundreds of different inputs. 

We created `ActionSet` to make this process as easy as possible on practitioners. `ActionSet` is designed to reduce the amount of information you have to enter by infering variable types and bounds from a sample of points.


In [4]:
A = rs.ActionSet(X) 
print(A)

+---------------------------------------+---------------+------------+-----+---------+----------------+-----------+-----------+
|                                  name | variable type | actionable |  lb |      ub | step direction | step type | step size |
+---------------------------------------+---------------+------------+-----+---------+----------------+-----------+-----------+
|                               Married |          bool |       True | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                                Single |          bool |       True | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                             Age_lt_25 |          bool |       True | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                       Age_in_25_to_40 |          bool |       True | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                       Age_in_40_to_59 |          bool |       True | 0.0 |     1.0 |              0 | 

Practitioners can then fine-tune the set of feasible actions for each variable with using a familiar pandas-like API.


In [5]:
# Variables that can't be changed
A['Married'].actionable = False
A[['Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60']].actionable = False
A[['TotalMonthsOverdue', 'TotalOverdueCounts', 'HistoryOfOverduePayments']].actionable = False
print(A)

# EducationLevel takes values from (0, 3), can be changed in increments of 1, and must increase
A['EducationLevel'].bounds = (0, 3)
A['EducationLevel'].step_size = 1
A['EducationLevel'].step_type = "absolute" 
A['EducationLevel'].step_direction = 1 

# TotalMonthsOverview takes values between 0 to 12, can only increase by 1 month, and must increase
A['TotalMonthsOverdue'].bounds = (0, 12)
A['TotalMonthsOverdue'].step_size = 1  
A['TotalMonthsOverdue'].step_type = "absolute"  

+---------------------------------------+---------------+------------+-----+---------+----------------+-----------+-----------+
|                                  name | variable type | actionable |  lb |      ub | step direction | step type | step size |
+---------------------------------------+---------------+------------+-----+---------+----------------+-----------+-----------+
|                               Married |          bool |      False | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                                Single |          bool |       True | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                             Age_lt_25 |          bool |      False | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                       Age_in_25_to_40 |          bool |      False | 0.0 |     1.0 |              0 |  absolute |       1.0 |
|                       Age_in_40_to_59 |          bool |      False | 0.0 |     1.0 |              0 | 

# Generating Actionable Adverse Action Notices with `Flipset`

When our model is deployed, it will deny loan to any person whos input variables `x` receive a predicted outcome of 0 -- i.e., `clf.predict(x) == 0`. Looks like this was the case for person i = 13

In [6]:
x = X.values[[13]]
print('yhat[i] = %d' % clf.predict(x)[0])

yhat[i] = 0


We can generate an actionable adverse action notice for person `i` using the `Flipset` class. A `Flipset` represents a set of actions that a person can take to change their prediction from 0 to 1.

We initialize `Flipset` using the input variables for person i `x`, the action set `A` and the model `clf`. 

In [7]:
fs = rs.Flipset(x, action_set = A, clf = clf)

We can then find actions that allow the person to flip their prediction using the populate method.


In [8]:
fs.populate(enumeration_type = 'mutually_exclusive', total_items = float('inf'));

recovered all minimum-cost items
obtained 3 items in 0.1 seconds



Populating a flipset is very fast! Here, we were able find 3 items. Each item reflects a different way that the person can flip their prediction. The items are required to change different variables. 

We can see each changes by using `print` or the `to_html()` function. 

In [9]:
display(HTML(fs.to_html()))

Features to Change,Current Value,to,Required Value
MaxBillAmountOverLast6Months,2060,→,5814
MonthsWithZeroBalanceOverLast6Months,0,→,1
MonthsWithHighSpendingOverLast6Months,4,→,0
MostRecentBillAmount,2010,→,0
MostRecentPaymentAmount,100,→,1365


Here, person `i` could have changed their prediction from 0 to 1 (and thus qualify for the loan) through the performing any of the following actions:

1. Increase `MaxBillAmountOverLast6Months` from \$2,060 to \$5,700

or 

2. Increase `MonthsWithZeroBalanceOverLast6Months` from 0 to 1, *and* Decrease `MonthsWithHighSpendingOverLast6Months` from 4 to 0, *and* Decrease `MostRecentBillAmount` from \$2,010 to \$535.

or 

3. Increase `MostRecentPaymentAmount` from \$100 to \$1,218.


## Empty Flipsets = No Recourse

The `populate` produce is designed to search over *all possible actions* that a person can perform. This is computationally hard as the number of possible actions increases exponentially. However, it is valuable because if we find that flipset is empty after calling populate, then this means that no possible action can flip person's prediction - i.e., **the person has no recourse.**

In what follows, we demonstrate how this can happen for person `i = 649`. Here, consider a more stringent model that will only approve applicants with a predicted probability of repayment greater than 95% (c.f., the previous model required a predicted probability of repayment of 50%). We can construct this model by shifting the intercept of the previous model by `np.log(0.95 / (1.0 - 0.95))).`

In [10]:
x = X.values[[649]]
fs = rs.Flipset(x, action_set = A, coefficients = clf.coef_[0], intercept = clf.intercept_[0] - np.log(0.95 / (1.0 - 0.95)))
fs.populate(enumeration_type = 'distinct_subsets', total_items = 1);
display(HTML(fs.to_html()))

recovered all minimum-cost items
obtained 0 items in 0.0 seconds


# Check that a Model Provides Recourse with `RecourseAuditor`

> Indented block



Ideally, we want to ensure that a model will provide every consumer with some way to change their prediction before we deploy it. One way to check this is to constuct `Flipsets` for each consumer in the training dataset. To make this process simpler, we produced `RecourseAuditor`

In [11]:
# Basic Recourse Verification with 1 Model
# Use the auditor on 100 points (live)
# It's super easy

# How many people are dnied
# How many have recourse?
# How difficult is that recourse?

from recourse import RecourseAuditor

ra = RecourseAuditor(
    action_set=A,
    coefficients=clf.coef_[0],
    intercept=clf.intercept_[0] - score_threshold,
    solver='python-mip'
)

audit_output = ra.audit(X.sample(100))

NameError: name 'score_threshold' is not defined

In [ ]:
audit_output['feasible'].value_counts()